# Day 22: Differential Privacy in FL

**Mathematical Privacy Guarantees for Federated Learning**

## Overview
- **Goal**: Guarantee that individual client data cannot be inferred
- **Method**: Add calibrated noise to gradient updates
- **Guarantee**: (ε, δ)-differential privacy

## What You'll Learn
1. **DP Definition**: Formal privacy guarantee
2. **DP-SGD**: Training with gradient noise
3. **Privacy Accounting**: Tracking privacy budget
4. **Utility-Privacy Tradeoff**: Noise vs accuracy

---

## 1. What is Differential Privacy?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("""
DIFFERENTIAL PRIVACY (DP):

Definition:
  A randomized algorithm M satisfies (ε, δ)-DP if:
    
    P[M(D) ∈ S] ≤ e^ε × P[M(D') ∈ S] + δ
    
    for all datasets D, D' differing in ONE element
    and all output sets S.

Intuition:
  • Output doesn't change MUCH if ONE data point changes
  • e^ε controls how much the output can change
  • δ is failure probability (usually δ ≪ 1/|D|)

Meaning:
  • Adversary CANNOT determine if YOUR data was used
  • Regardless of auxiliary information
  • Mathematical guarantee, not heuristic

Parameters:
  • ε (epsilon): Privacy loss
    - ε = 0: Perfect privacy (no information leaked)
    - ε = 1: Reasonable privacy
    - ε = 10: Weak privacy (some information leaked)
    
  • δ (delta): Failure probability
    - Typically δ = 10^-5 or smaller
    - Probability that DP guarantee fails

In Federated Learning:
  • Add noise to client updates BEFORE sending
  • Server sees: gradient + noise
  • Privacy: Individual update protected by noise

""")

## 2. DP-SGD: Training with Gradient Noise

In [ ]:
def clip_gradient(grad, norm_bound):
    """
    Clip gradient to bound its L2 norm.
    
    Why clip?
    - Limits sensitivity of each update
    - Required for DP (bounded sensitivity)
    """
    grad_norm = np.linalg.norm(grad)
    if grad_norm > norm_bound:
        grad = grad * (norm_bound / grad_norm)
    return grad

def add_dp_noise(grad, noise_multiplier, norm_bound):
    """
    Add Gaussian noise for DP-SGD.
    
    Args:
        grad: Gradient vector
        noise_multiplier: σ (controls privacy)
        norm_bound: Clipping bound (C)
        
    Returns:
        Noisy gradient
    """
    # Clip gradient
    clipped_grad = clip_gradient(grad, norm_bound)
    
    # Add Gaussian noise: N(0, σ²C²)
    noise = np.random.randn(*grad.shape()) * noise_multiplier * norm_bound
    
    return clipped_grad + noise

# Example
gradient = np.random.randn(100) * 0.5

# Without DP
clean_update = gradient

# With DP (different noise levels)
dp_update_1 = add_dp_noise(gradient, noise_multiplier=0.1, norm_bound=1.0)
dp_update_2 = add_dp_noise(gradient, noise_multiplier=1.0, norm_bound=1.0)
dp_update_3 = add_dp_noise(gradient, noise_multiplier=5.0, norm_bound=1.0)

# Visualize
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

updates = [clean_update, dp_update_1, dp_update_2, dp_update_3]
titles = ['No DP', 'DP σ=0.1', 'DP σ=1.0', 'DP σ=5.0']

for ax, update, title in zip(axes, updates, titles):
    ax.plot(update, alpha=0.7)
    ax.set_title(title)
    ax.set_xlabel('Parameter Index')
    ax.set_ylabel('Value')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Observation:")
print("  • Higher noise multiplier = more privacy but less utility")
print("  • σ=0.1: Barely visible noise (weak privacy)")
print("  • σ=1.0: Moderate noise (reasonable privacy)")
print("  • σ=5.0: Heavy noise (strong privacy, poor utility)")

## 3. Privacy Accounting

In [ ]:
def compute_dp_epsilon(
    noise_multiplier,
    n_rounds,
    batch_size,
    n_samples,
    delta=1e-5
):
    """
    Compute ε for DP-SGD (simplified).
    
    Uses moments accountant approach (simplified formula).
    
    ε = noise_multiplier^-2 × 2 × log(1.25/delta) × rounds
    
    (This is simplified; real implementations use RDP accountant)
    """
    # Simplified moments accountant
    steps = n_rounds * (n_samples / batch_size)
    epsilon = (steps * (batch_size / n_samples) ** 2) / (noise_multiplier ** 2)
    
    return epsilon

# Privacy budget tracking
scenarios = [
    ('Weak Privacy', 0.5, 30),
    ('Moderate Privacy', 2.0, 30),
    ('Strong Privacy', 10.0, 30),
]

print("\n" + "="*60)
print("PRIVACY BUDGET ACCOUNTING")
print("="*60)

for name, noise, rounds in scenarios:
    epsilon = compute_dp_epsilon(noise, rounds, batch_size=32, n_samples=10000)
    print(f"\n{name}:")
    print(f"  Noise multiplier: {noise}")
    print(f"  Rounds: {rounds}")
    print(f"  ε (privacy loss): {epsilon:.2f}")
    
    if epsilon < 1:
        print(f"  → Strong privacy (ε < 1)")
    elif epsilon < 10:
        print(f"  → Reasonable privacy (1 < ε < 10)")
    else:
        print(f"  → Weak privacy (ε > 10)")

## 4. Utility-Privacy Tradeoff

In [ ]:
# Simulate training with different DP levels
noise_levels = [0.1, 0.5, 1.0, 2.0, 5.0, 10.0]
final_accuracies = []
epsilons = []

for noise in noise_levels:
    # Simulate FL with DP (simplified)
    accuracy = 0.90 / (1 + 0.1 * noise)  # Degraded by noise
    final_accuracies.append(accuracy)
    
    epsilon = compute_dp_epsilon(noise, 30, 32, 10000)
    epsilons.append(epsilon)

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy vs noise
axes[0].plot(noise_levels, final_accuracies, 'o-', linewidth=2, markersize=6, color='steelblue')
axes[0].set_xlabel('Noise Multiplier (σ)', fontsize=12)
axes[0].set_ylabel('Final Accuracy', fontsize=12)
axes[0].set_title('Utility vs Noise Level', fontsize=14)
axes[0].grid(True, alpha=0.3)

# Accuracy vs epsilon
axes[1].plot(epsilons, final_accuracies, 'o-', linewidth=2, markersize=6, color='coral')
axes[1].set_xlabel('Privacy Loss (ε)', fontsize=12)
axes[1].set_ylabel('Final Accuracy', fontsize=12)
axes[1].set_title('Utility-Privacy Tradeoff', fontsize=14)
axes[1].axvline(x=1, color='green', linestyle='--', alpha=0.5, label='ε=1 (strong)')
axes[1].axvline(x=10, color='orange', linestyle='--', alpha=0.5, label='ε=10 (weak)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nTradeoff Summary:")
print("  • More noise = Better privacy (lower ε) but worse accuracy")
print("  • Less noise = Better accuracy but weaker privacy")
print("  • Target: ε < 10 with acceptable accuracy loss")

## 5. Summary

### Differential Privacy Summary:

**Key Concepts:**

1. **(ε, δ)-DP Guarantee**:
   - Mathematical proof of privacy
   - ε: privacy loss parameter
   - δ: failure probability

2. **DP-SGD Algorithm**:
   - Clip gradients to bound sensitivity
   - Add Gaussian noise: N(0, σ²C²)
   - Track privacy budget across rounds

3. **Privacy Accounting**:
   - Each round consumes privacy budget
   - Use moments accountant or RDP
   - Stop when ε exceeds target

**Tradeoff:**
- Strong privacy (low ε): High noise, lower accuracy
- Weak privacy (high ε): Low noise, higher accuracy
- Target: ε < 10 with <5% accuracy loss

**When to Use DP:**
- ✅ High privacy requirements (healthcare, finance)
- ✅ Regulatory compliance (GDPR, HIPAA)
- ✅ Strong threat model (curious server)

**When NOT to Use DP:**
- ❌ Accuracy is critical
- ❌ Limited training budget
- ❌ Weak threat model

### Next Steps:
→ **Day 23**: Secure Aggregation (cryptography-based privacy)
→ **Day 25**: Membership Inference Attack (why we need DP)

---

**📁 Project Location**: `02_federated_learning_foundations/dp_federated_learning/`